<a href="https://colab.research.google.com/github/peter000815/BA0815/blob/master/download_financial_statement%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct  6 15:16:44 2021

@author: pro
"""

from urllib import request
import requests
import os
import openpyxl


print(os.getcwd())

def getKeyList(file):

    x = os.getcwd()

    wb = openpyxl.load_workbook(file)
    print(wb, type(wb))
    print(wb.sheetnames)

    sheet = wb.active
    minrow = sheet.min_row  
    maxrow = sheet.max_row
    mincol = sheet.min_column
    maxcol = sheet.max_column
    print('该文件', maxrow, '行', maxcol, '列')

    key_list = []
    for i in range(minrow+1, maxrow+1):
        point = 'A' + str(i)
        person = sheet[point].value
        if person:
            num = person.split("=")[-1].replace('"', "")
            key_list.append(num)
    key_list.sort()
    return key_list


def stock(key):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }  #这里找浏览器，按F12之后按ctrl+r，然后随便找一个js文件，点开后复制其中的User-Agent
    url1 = f"http://www.cninfo.com.cn/new/information/topSearch/query?keyWord={key}&maxNum=10"
    orgId = requests.post(url1, headers=headers).json()[0]["orgId"]
    url = 'http://www.cninfo.com.cn/new/hisAnnouncement/query'
    data = {
        'pageNum':'1',
        'pageSize': '30',
        'column': 'szse',
        'tabName':'fulltext',
        'stock': key+','+orgId, 
        'category': 'category_ndbg_szsh',#category_scgkfx_szsh（首次公开发行及上市）、category_ndbg_szsh（年度报告）、category_bndbg_szsh（半年度报告）
        'seDate': '2016-05-01~2019-06-01', #选择需要下载报告的时间
        'isHLtitle': 'true',
    }

    dict_data = requests.post(url, data=data, headers=headers).json()
    if dict_data["totalRecordNum"] < 1:
        print(key, "无年报数据")
        print(dict_data)
        return False
    for value_dict in dict_data["announcements"]:
            riqi = value_dict["announcementId"]
            name = value_dict["announcementTitle"]
            wjjname1 = value_dict["secName"]
            wjjname = wjjname1.replace("*", "star")

            if not os.path.exists(wjjname + key):
                os.mkdir(wjjname + key)
                print(wjjname + key + '创建成功')

            if name.find('摘要') >= 1:
                continue
            else:  
                url_r = 'http://www.cninfo.com.cn/new/announcement/download?bulletinId='+riqi
                request.urlretrieve(url_r, wjjname + key + '/' + wjjname + key + '--' + name + '.pdf')
                print(wjjname + name + '保存成功')

if __name__ == '__main__':
    key = input('请输入股票代码:')
    stock(key)    
    #key_list = getKeyList("data.xlsx")  # 可以用表格导入代码
    #print(key_list)

    #for i in key_list:
        #stock(i)

